In [1]:
import dash
import pandas as pd
import plotly.express as px
from dash import html
from dash import dcc
from dash.dependencies import Input, Output

In [2]:
df = pd.read_csv('cc_data.csv')
df[:10]
#d = ['2021-12-08','2021-12-07','2021-12-06']
#dft = df[df['Date'].isin(d)]
l = [0,1,3,4,5]
dfM = df.iloc[:,l]
dfM = pd.melt(dfM, id_vars=['Date','Date_Time','Diff'],var_name='hash', value_name='rate')
dfM[dfM['Date']=='2021-11-21']

,Date,Date_Time,Diff,hash,rate
1,2021-11-21,13:33:51.397040,0.000000,Reported_Hash,30
2,2021-11-21,13:34:53.458088,0.000000,Reported_Hash,30
3,2021-11-21,14:01:40.669087,350.001428,Reported_Hash,30
4,2021-11-21,14:03:51.583825,141.313850,Reported_Hash,30
5,2021-11-21,14:04:12.533754,0.000000,Reported_Hash,30
6,2021-11-21,14:04:37.615924,147.833664,Reported_Hash,30
7,2021-11-21,14:11:19.639080,0.000000,Reported_Hash,30
8,2021-11-21,17:31:46.397327,2769.516563,Reported_Hash,30
9,2021-11-21,17:33:12.344673,312.401187,Reported_Hash,30
10,2021-11-21,17:53:13.076384,624.334026,Reported_Hash,30


In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div([ #main div
    html.Div(children=[ #Graph, Dropdown and Radio Div
    dcc.Graph(id='graph-with-drop'), 
    dcc.Dropdown(
        id='drop-year',
        options=[{'label':i,'value':i} for i in dfM['Date'].unique()],value=dfM['Date'].max()
    ), 
    dcc.RadioItems(
        id='radio_coin_hash',
        options=[
            {'label':'Hash Rate','value':'rate'},
            {'label':'Coins','value':'Diff'}
        ],value='rate'#CLOSE RADIO list
    ), #CLOSE RADIO button
        html.Br(),
        ]), #CLOSE GRAPH, DROPDOWN and RADIO Div
    
    html.Div(children=[ #reported hash pie div
        html.H1('Current Hash Rate'),
        dcc.Graph(id='reported_hash_pie')
    ],style={'padding': 10, 'flex': 1, 'display': 'inline-block', 'margin-left':'75px'}), #CLOSE REPORTED HASH div
    
    html.Div(children=[ #calc hash pie div
        html.H1('Calculated Hash Rate'),
        dcc.Graph(id='calc_hash_pie')
    ],style={'padding': 10, 'flex': 1, 'display': 'inline-block', 'margin-left':'250px'}) #CLOSE CALC HASH div
    
]) #close main div

@app.callback(
    Output('graph-with-drop', 'figure'),
    Input('drop-year', 'value'),
    Input('radio_coin_hash', 'value'))
def update_figure(selected_year,cat):
    filtered_df = dfM[dfM.Date == selected_year]
    fig = px.bar(filtered_df, x="Date_Time", y=cat,
                    color="hash", barmode='group')
    fig.update_layout(transition_duration=500)
    return fig

@app.callback(
    Output('reported_hash_pie', 'figure'),
    Input('drop-year', 'value'))
def update_figure(selected_year):
    dfRH = dfM[(dfM['Date']==dfM['Date'].max()) & (dfM['hash']=='Reported_Hash')]
    rh = dfRH[dfRH['Date_Time']==dfRH['Date_Time'].max()]['rate']

    labels = pd.Series([int(rh),'..'])
    values = pd.Series([int(rh),100-int(rh)])

    f = {'Label':labels, 'Value':values}
    df = pd.DataFrame(f)

    fig = px.pie(df,values='Value',names='Label')
    fig.update_traces(textinfo='label', hole=.5, textfont_size=18)
    fig.update_layout(annotations=[dict(text='Current_Hash_Rate',showarrow=False, font_size=20)]) 

    return fig

@app.callback(
    Output('calc_hash_pie', 'figure'),
    Input('drop-year', 'value'))
def update_figure(selected_year):
    dfRH = dfM[(dfM['Date']==dfM['Date'].max()) & (dfM['hash']=='Calculated_Hash')]
    ch = dfRH[dfRH['Date_Time']==dfRH['Date_Time'].max()]['rate']

    labels = pd.Series([int(ch),'..'])
    values = pd.Series([int(ch),100-int(ch)])

    f = {'Label':labels, 'Value':values}
    df = pd.DataFrame(f)

    fig = px.pie(df,values='Value',names='Label')
    fig.update_traces(textinfo='label', hole=.5, textfont_size=18)
    fig.update_layout(annotations=[dict(text='Calculated_Hash_Rate',showarrow=False, font_size=20)]) 

    return fig


if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Dec/2021 15:40:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2021 15:40:15] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2021 15:40:15] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2021 15:40:15] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Dec/2021 15:40:15] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Dec/2021 15:40:15] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Dec/2021 15:40:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2021 15:40:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2021 15:40:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2021 15:40:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2021 15:40:51] "POST /_dash-update-component HTTP/1.

In [ ]:
#Current_Hash=dfM['rate'][-1:]
#Current_Hash

#dfRH = dfM[(dfM['Date']==dfM['Date'].max()) & (dfM['hash']=='Reported_Hash')]
#rh = dfRH[dfRH['Date_Time']==dfRH['Date_Time'].max()]['rate']
#dfM[(dfM['Date_Time']==dfM['Date_Time'].max()) & (dfM['Date']==dfM['Date'].max())]
#dfM['Date_Time']==dfM['Date_Time'].max()